In [1]:
import pandas as pd
import numpy as np

from dateutil.relativedelta import relativedelta

from datetime import *

# I. Processing

## Import loan book as a dataframe

In [2]:
loan_book_df = pd.read_csv('Loan_Book.csv', delimiter='\t')

loan_book_df.head()

,LoanID,BorrowerID,Day of StartDate,Term,Day of MaturityDate,Security,ShieldCoverage,LoanPurpose,LoanStatus,RepaymentStatus,RepaymentType,default_reason,StartDate,Amount,GrossRate,grossrate,PrincipalOutstanding,prinipaloutstanding,term
0,42371,4427454,8 November 2021,36 months,8 November 2024,Unsecured,Yes,Personal Loan,Active,Repaying,Amortising,NaN,08/11/2021,21000.0,0.04,4.00157,20480.46,20480.46,36
1,42241,4425173,3 November 2021,24 months,14 December 2021,Unsecured,Yes,Personal Loan,Settled,Repaid,Amortising,NaN,03/11/2021,1070.0,0.04,4.00157,0.00,0.00,24
2,42265,4428233,3 November 2021,60 months,20 November 2026,Unsecured,Yes,Personal Loan,Active,Repaying,Amortising,NaN,03/11/2021,5250.0,0.04,4.00157,5190.02,5190.02,60
3,42213,3503532,2 November 2021,60 months,2 November 2026,Unsecured,Yes,Personal Loan,Active,Repaying,Amortising,NaN,02/11/2021,5250.0,0.04,4.00157,5176.92,5176.92,60
4,42216,3829305,2 November 2021,60 months,2 November 2026,Unsecured,Yes,Personal Loan,Active,Repaying,Amortising,NaN,02/11/2021,8925.0,0.04,4.00157,8801.00,8801.00,60


## Organising Data

### 1. Remove redundant columns in loan book 
_StartDate (str), Maturity Date (str), Term (str), Security, Shield Coverage, RepaymentType, , GrossRate (float, 2.sf), principaloutstanding_

In [3]:
loans_df = loan_book_df.loc[:, ~loan_book_df.columns.isin(['Day of StartDate','Term','Day of MaturityDate', 'Security', 'ShieldCoverage','RepaymentType','prinipaloutstanding', 'GrossRate'])]

### 2. Group loans by their borrowers

This section was carried out in part II "Data Cleansing" to make borrowers records accurate

### 3. Maturity date of loans (datetime format)

In [4]:
loans_df.loc[:, 'MaturityDate'] = pd.to_datetime(loan_book_df['Day of MaturityDate'], dayfirst=True)
loans_df.head(5)

C:\Users\arsia\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\arsia\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,LoanID,BorrowerID,LoanPurpose,LoanStatus,RepaymentStatus,default_reason,StartDate,Amount,grossrate,PrincipalOutstanding,term,MaturityDate
0,42371,4427454,Personal Loan,Active,Repaying,NaN,08/11/2021,21000.0,4.00157,20480.46,36,2024-11-08
1,42241,4425173,Personal Loan,Settled,Repaid,NaN,03/11/2021,1070.0,4.00157,0.00,24,2021-12-14
2,42265,4428233,Personal Loan,Active,Repaying,NaN,03/11/2021,5250.0,4.00157,5190.02,60,2026-11-20
3,42213,3503532,Personal Loan,Active,Repaying,NaN,02/11/2021,5250.0,4.00157,5176.92,60,2026-11-02
4,42216,3829305,Personal Loan,Active,Repaying,NaN,02/11/2021,8925.0,4.00157,8801.00,60,2026-11-02


### 4. Delete loan_book_df to reduce memory usage

In [5]:
del loan_book_df

# II. Data Cleansing

## 1. Incomplete entries, missing data, string spelling

- No missing values (NaN) in columns other than default_reason

### a) Defaulted loans with no default reason given

In [6]:
loans_df.loc[(loans_df['default_reason'].isnull())  & (loans_df['LoanStatus']=="Defaulted")][['LoanID','LoanStatus','default_reason']].head(5)

,LoanID,LoanStatus,default_reason
98,41263,Defaulted,NaN
622,40736,Defaulted,NaN
659,40676,Defaulted,NaN
850,40515,Defaulted,NaN
883,40488,Defaulted,NaN


In [7]:
loans_df.loc[(loans_df['default_reason'].isnull())  & (loans_df['LoanStatus']=="Defaulted"),"default_reason"] = "Unspecified"
loans_df.loc[(loans_df['default_reason']=="Unspecified")  & (loans_df['LoanStatus']=="Defaulted")][['LoanID','LoanStatus','default_reason']].head(5)

,LoanID,LoanStatus,default_reason
98,41263,Defaulted,Unspecified
622,40736,Defaulted,Unspecified
659,40676,Defaulted,Unspecified
850,40515,Defaulted,Unspecified
883,40488,Defaulted,Unspecified


### b) Spelling correction for Partial Settlements

In [8]:
loans_df.loc[loans_df['default_reason']=="artial Settlement"][['LoanID','LoanStatus','default_reason']].head(3)

,LoanID,LoanStatus,default_reason
522,40833,Defaulted,artial Settlement
12847,28668,Defaulted,artial Settlement
12848,2382,Defaulted,artial Settlement


In [9]:
loans_df.loc[loans_df['default_reason']=="artial Settlement", "default_reason"] = "Partial Settlement"

### c) Remove duplicates

In [10]:
loans_df[loans_df.duplicated('LoanID', keep=False)] # No duplicated loans with the same ID

loans_df[loans_df.duplicated(keep=False)] # No duplicated entries

,LoanID,BorrowerID,LoanPurpose,LoanStatus,RepaymentStatus,default_reason,StartDate,Amount,grossrate,PrincipalOutstanding,term,MaturityDate


## 2. Converting columns to appropriate data types

In [11]:
loans_df.dtypes ## Only the "StartDate" column needs to be converted to datetime

LoanID                           int64
BorrowerID                       int64
LoanPurpose                     object
LoanStatus                      object
RepaymentStatus                 object
default_reason                  object
StartDate                       object
Amount                         float64
grossrate                      float64
PrincipalOutstanding           float64
term                             int64
MaturityDate            datetime64[ns]
dtype: object

### a) StartDate of Loans

In [12]:
loans_df.head(1)

,LoanID,BorrowerID,LoanPurpose,LoanStatus,RepaymentStatus,default_reason,StartDate,Amount,grossrate,PrincipalOutstanding,term,MaturityDate
0,42371,4427454,Personal Loan,Active,Repaying,NaN,08/11/2021,21000.0,4.00157,20480.46,36,2024-11-08


In [13]:
loans_df['StartDate']= pd.to_datetime(loans_df['StartDate'], dayfirst=True)

In [14]:
loans_df.head(1)

,LoanID,BorrowerID,LoanPurpose,LoanStatus,RepaymentStatus,default_reason,StartDate,Amount,grossrate,PrincipalOutstanding,term,MaturityDate
0,42371,4427454,Personal Loan,Active,Repaying,NaN,2021-11-08,21000.0,4.00157,20480.46,36,2024-11-08


## 3. Calculating number of days that loans were paid off/defaulted earlier than intended maturity date

In [15]:
def get_days_early(start, end, term):
    mat_date = start + relativedelta(months=term)
    
    return (mat_date - end).days

loans_df.loc[loans_df['LoanStatus'] != "Active", 'DaysEarly'] = loans_df.apply(lambda row: get_days_early(row['StartDate'], row['MaturityDate'], row['term']), axis=1)

loans_df.loc[loans_df['LoanID']==23734]

,LoanID,BorrowerID,LoanPurpose,LoanStatus,RepaymentStatus,default_reason,StartDate,Amount,grossrate,PrincipalOutstanding,term,MaturityDate,DaysEarly
33360,23734,946848,Sole Trader,Defaulted,Defaulted,Non-Payment,2018-12-14,5278.0,5.0011,2883.57,36,2021-03-31,258.0


## 4. Inaccuracies

### a) Settled loans with outstanding principal balances

Could be due to paying off debts earlier than loan's term, so outstanding balance was not updated. Could also be due to debt forgiveness.

In [16]:
loans_df.loc[(loans_df['LoanStatus']=="Settled") & (loans_df['PrincipalOutstanding']>0)][['LoanID','LoanStatus','PrincipalOutstanding','DaysEarly']].head(5)

,LoanID,LoanStatus,PrincipalOutstanding,DaysEarly
31,41350,Settled,2000.87,234.0
76,41277,Settled,3993.30,1319.0
169,41190,Settled,6649.67,953.0
287,41053,Settled,1988.28,1661.0
349,41005,Settled,11826.68,935.0


In [17]:
loans_df.loc[(loans_df['LoanStatus']=="Settled") & (loans_df['PrincipalOutstanding']>0),'PrincipalOutstanding'] = 0

## 5. Group loans for each borrower and create borrowers dataframe
Store aggregated:
- sums of borrowed funds and outstanding balance in debit
- number of active, settled, and defaulted loans
- number of loans with delinquent payments

In [18]:
borrowers_df = loans_df.groupby('BorrowerID').agg(total_loans = ('LoanID', 'count'),
                                                  total_borrowed = ('Amount','sum'),
                                                  balance = ('PrincipalOutstanding','sum'),
                                                  active = ('LoanStatus', lambda x: x[x.str.contains('Active')].count()),
                                                  settled = ('LoanStatus', lambda x: x[x.str.contains('Settled')].count()),
                                                  defaults = ('LoanStatus', lambda x: x[x.str.contains('Defaulted')].count()),
                                                  delinquencies = ('RepaymentStatus', lambda x: x[x.str.contains('Late')].count())).reset_index()

borrowers_df.dtypes

BorrowerID          int64
total_loans         int64
total_borrowed    float64
balance           float64
active              int64
settled             int64
defaults            int64
delinquencies       int64
dtype: object

## Export new dataframes to seperate CSV files

In [19]:
loans_df.to_csv('Loans_Data.csv', sep="\t", index=False)
borrowers_df.to_csv('Borrowers_Data.csv', sep="\t", index=False)